

# Projeto de Aprendizado de Máquina Aplicado à Robótica
## Equipe: 
- Bruno de Oliveira Pinheiro Júnior (bopj)
- Daniel Victor da Costa Carneiro Salvador (dvccs)
- Estela de Andrade Joffily (eaj2)

> --Esta célula pode ser apagada depois, é apenas para consultar o enunciado--

**(DONE)** Item 1. Escolha uma base de dados (por exemplo do repositório UCI) de um problema de classificação ou regressão. Apresente o problema de forma suscinta, com suas variáveis
preditoras e alvo.

**(DONE)** Item 2. Aplique o algoritmo de árvores de decisão e inspecione o conhecimento adquirido. Vc pode por exemplo selecionar a partir da árvore construída, regras com boa cobertura e confiança e discuti-las.

**(DONE)** Item 3. Realize experimentos com algoritmos diversos e selecione o melhor algoritmo com base em uma métrica de avaliação de interesse. Justifique a escolha da métrica. Dependendo do algoritmo, faça experimentos com variação de parâmetros (e.g., valor do parâmetro k, do algoritmo kNN).

**(IN PROGRESS)** Item 4. Escolha um ou mais algoritmos de classificação (por exemplo, IBk) e use técnicas de seleção de atributos para identificar as variáveis consideradas mais relevantes para o problema de classificação escolhido. Analise o impacto da seleção de atributos, avaliando o algoritmo escolhido usando (a) todos os atributos do problema, ou seja, sem seleção (b) e apenas usando os atributos selecionados.

Item 5. Selecione o melhor modelo encontrado no item anterior e aplique pelo menos duas técnicas de interpretabilidade, como:

* Feature importante: discuta que características foram mais importantes para o modelo;

* Global surrogate: apresente o modelo interpretável gerado e discuta os insights observados com o modelo;  

* Local surrogate: apresente pelo menos dois exemplos a serem explicados e gere as explicações com o LIME;

* PDP: faça pelos menos três gráficos PDP, variando os atributos analisados e discuta os resultados; ou outras técnicas de interesse.

Discuta os insights e conclusões sobre a aplicação das técnicas de interpretabilidade aplicadas.

A célula abaixo realizará os imports necessários para a execução deste notebook. Os modelos utilizados serão do Scikit Learn enquanto que usaremos principalmente o Seaborn para plotar gráficos e as estatísticas necessárias. O pandas será usado para manipular o dataframe.

In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import learning_curve, LearningCurveDisplay
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.inspection import PartialDependenceDisplay

Vamos agora garantir a existência de uma pasta **plots** para armazenar os gráficos gerados durante a sessão.

In [27]:
cwd = Path.cwd()
Path(str(cwd)+"/plots").mkdir(parents=True, exist_ok=True)

### Apresentação do *dataset* e preparação inicial

<img src="pictures/galaxy.jpg" width="500" title="Galaxy"> <img src="pictures/quasar.jpg" width="400" title="Quasar"> <img src="pictures/star.jpg" width="400" title="Star">

O [Stellar Classification Dataset - SDSS17](https://www.kaggle.com/datasets/fedesoriano/stellar-classification-dataset-sdss17?resource=download) é uma base de dados que classifica objetos estelares com base em suas características espectrais. Este *dataset* contém 100.000 instâncias classificadas dentre estrelas, galáxias e quasares (núcleos luminosos de galaxias alimentados por buracos negros supermassivos), provenientes de observações do SDSS (**Sloan Digital Sky Survey**). Cada instância tem 17 atributos e a coluna `class` que se refere à classe:

- `obj_ID` = **Object Identifier**, valor único que identifica o objeto no catálogo de imagens usado pelo CAS
- `alpha` = Ângulo de ascensão reto (à época J2000)
- `delta` = Ângulo de declinação (à época J2000)
- `u` = Filtro ultravioleta no sistema fotométrico
- `g` = Filtro verde no sistema fotométrico
- `r` = Filtro vermelho no sistema fotométrico
- `i` = Filtro infravermelho próximo no sistema fotométrico
- `z` = Filtro infravermelho no sistema fotométrico
- `run_ID` = Número de execução usado para identificar a varredura específica
- `rereun_ID` = Número de repetição da execução para especificar como a imagem foi processada
- `cam_col` = Coluna da câmera para identificar a linha de varredura dentro da execução
- `field_ID` = Número do campo para identificar cada campo
- `spec_obj_ID` = ID única utilizada por objetos espectroscópicos ópticos (2 observações diferentes com o mesmo `spec_obj_ID must` são da mesma classe)
- `class` = classe do objeto ('galaxy', 'star' ou 'quasar')
- `redshift` = Valor do desvio para vermelho com base no aumento do comprimento de onda
- `plate` = ID da placa, identifica cada placa no SDSS
- `MJD` = Data Juliana Modificada, usada para indicar quando um dado do SDSS foi obtido
- `fiber_ID` = ID da fibra, identifica a fibra que apontou a luz para o plano focal em cada observação

In [28]:
data = pd.read_csv('star_classification.csv')
data.head()

,obj_ID,alpha,delta,u,g,r,i,z,run_ID,rerun_ID,cam_col,field_ID,spec_obj_ID,class,redshift,plate,MJD,fiber_ID
0,1.237661e+18,135.689107,32.494632,23.87882,22.27530,20.39501,19.16573,18.79371,3606,301,2,79,6.543777e+18,GALAXY,0.634794,5812,56354,171
1,1.237665e+18,144.826101,31.274185,24.77759,22.83188,22.58444,21.16812,21.61427,4518,301,5,119,1.176014e+19,GALAXY,0.779136,10445,58158,427
2,1.237661e+18,142.188790,35.582444,25.26307,22.66389,20.60976,19.34857,18.94827,3606,301,2,120,5.152200e+18,GALAXY,0.644195,4576,55592,299
3,1.237663e+18,338.741038,-0.402828,22.13682,23.77656,21.61162,20.50454,19.25010,4192,301,3,214,1.030107e+19,GALAXY,0.932346,9149,58039,775
4,1.237680e+18,345.282593,21.183866,19.43718,17.58028,16.49747,15.97711,15.54461,8102,301,3,137,6.891865e+18,GALAXY,0.116123,6121,56187,842


A célula abaixo vai remover do dataframe colunas que identificavam, não o objeto astronômico, mas sim seu processo de detecção, sendo irrelevante para a predição que vamos realizar. Além disso, vamos construir o array X de atributos e o array y com os alvos para nossos modelos e realizar a normalização dos nossos dados:

In [29]:
# Limpando o dataframe
id_columns = ['obj_ID', 'run_ID', 'rerun_ID', 'cam_col',
              'field_ID', 'spec_obj_ID', 'plate', 'MJD',
              'fiber_ID']
df = data.drop(columns = id_columns, axis=1)

# Alvos e atributos
y = df['class']
X = df.drop(columns = ['class'], axis=1)

# Normalização do dataset pela média e desvio padrão
X = (X - X.mean())/X.std()

# Formato das matrizes e proporção entre as classes
X.shape, y.shape
print(f"Shape of X: {X.shape}\nShape of Y: {y.shape}\n")
print(df['class'].value_counts(normalize=True))
X.head()


Shape of X: (100000, 8)
Shape of Y: (100000,)

class
GALAXY    0.59445
STAR      0.21594
QSO       0.18961
Name: proportion, dtype: float64


,alpha,delta,u,g,r,i,z,redshift
0,-0.434601,0.425527,0.059754,0.054926,0.403960,0.046007,0.003937,0.079557
1,-0.339920,0.363400,0.088045,0.072456,1.584398,1.185091,0.092834,0.277095
2,-0.367249,0.582710,0.103326,0.067165,0.519743,0.150018,0.008808,0.092422
3,1.669515,-1.249099,0.004921,0.102209,1.059899,0.807606,0.018321,0.486768
4,1.737301,-0.150241,-0.080055,-0.092947,-1.697412,-1.767878,-0.098468,-0.630263


Vamos realizar a divisão do dataset em treino e teste, numa proporção de 80% para treino e 20% para teste. O parâmetro `random_state` é a semente do gerador de números aleatórios:

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Antes de começarmos o treinamento dos modelos, vamos criar uma função para receber um vetor com as predições dos modelos e um vetor com os alvos, para recebermos um relatório com algumas métricas que indicarão o desempenho do algoritmo, a saber: 
- **Acurácia**: representa a fração de predições corretas feitas pelo modelo.
- **Precisão**: representa a fração de predições positivas corretas feitas pelo modelo.
- **Recall**: representa a fração de positivos corretamente identificados pelo modelo.
- **F-score**: média harmônica entre precisão e recall.

Além disso veremos também a matriz de confusão:

In [31]:
def get_metrics(model_name ,y_test, y_pred):
    """
    Returns the metrics of a given prediction


    Args:
        y_test (array): Test targets
        y_pred (array): Test predictions
    """
    accuracy = metrics.accuracy_score(y_test, y_pred, normalize=True)
    precision = metrics.precision_score(y_test, y_pred, average='macro')
    recall = metrics.recall_score(y_test, y_pred, average='macro')
    f1_score = metrics.f1_score(y_test, y_pred, average='macro')

    results_dict = {
        'model_name': model_name,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1_score,
    }

    return results_dict


def tree_model_report(y_test, y_pred, savename='tree_report'):
    """

    Args:
        y_test (list): Test targets
        y_pred (list): Test predictions
        savename (str): Name to be used when saving report image
    """
    matrix = metrics.confusion_matrix(y_test, y_pred, normalize='true')
    results_dict = get_metrics('D' ,y_test, y_pred)
    report = (f'Accuracy: {results_dict['accuracy']:.2f}\n'
              f'Precision: {results_dict['precision']:.2f}\n'
              f'Recall: {results_dict['recall']:.2f}\n'
              f'F-Score: {results_dict['f1_score']:.2f}\n'
            )

    sns.heatmap(matrix, annot=True)
    plt.title(report)
    plt.savefig(f'plots/{savename}.png', bbox_inches='tight', dpi=300)
    plt.close()


### Aplicando um algoritmo de árvore de decisão

Vamos começar com modelos de **árvores de decisão**. Estes são modelos de classificação ou regressão que dividem os dados em subconjuntos com base em condições nos atributos, formando uma estrutura em forma de árvore. As folhas representam as decisões finais, e os nós internos representam testes nos atributos. Vamos usar como parâmetros para esses modelos:
 - `criterion`: determina como a árvore escolhe onde dividir os dados.
 - `max_depth`: profundidade máxima da árvore.
 - `random_state`: semente do gerador de números aleatórios. Importante para garantir a reprodutibilidade do experimento.
 - `min_samples_leaf`: número mínimo de amostras que um nó folha deve conter. Ajuda a evitar a criação de folhas muito pequenas e, assim, controla o overfitting.
 - `ccp_alpha`: parâmetro de poda de custo-complexidade. Ajuda a simplificar a árvore, removendo nós cuja complexidade não justifica o ganho em desempenho, evitando overfitting.

 Criaremos uma função para instanciar diversos modelos, variando o parâmetro de profundidade:

In [32]:
def decision_tree_classifier_evaluate_depth(
        min_depth: int,
        max_depth: int,
        min_samples_leaf: int = 1,
):
    """
    Function to evaluate the Decision Tree Classifier with different depths.

    Args:
        min_depth (int): The range minimum depth
        max_depth (int): The range maximum depth
        min_samples_leaf (int, optional): Defaults to 1.
    """
    for i in range(min_depth, max_depth+1):
        model = DecisionTreeClassifier(
            criterion='entropy',
            max_depth=i,
            random_state=42,
            min_samples_leaf=min_samples_leaf,
            ccp_alpha=0.0
        )

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        tree_model_report(y_test, y_pred, savename=f'min_depth_{i}_tree')


Aplicando o algoritmo:

In [33]:
decision_tree_classifier_evaluate_depth(min_depth=2, max_depth=5)

**Tree Depth = 2**: <img src="plots/min_depth_2_tree.png" width="600" title="Tree Depth = 2"> **Tree Depth = 3**: <img src="plots/min_depth_3_tree.png" width="600" title="Tree Depth = 3">

**Tree Depth = 4**: <img src="plots/min_depth_4_tree.png" width="600" title="Tree Depth = 4"> **Tree Depth = 5**: <img src="plots/min_depth_5_tree.png" width="600" title="Tree Depth = 5">


Analisando as métricas, vamos utilizar a árvore com profundidade máxima 4. Vamos instânciá-la novamente e mostrar sua estrutura:

In [34]:
tree_model = DecisionTreeClassifier(
    criterion='entropy',
    max_depth=4,
    min_samples_leaf=1,
    random_state=42,
    ccp_alpha=0.0
)

tree_model.fit(X_train, y_train)

plt.figure(figsize=(36, 18))
plot_tree(
    tree_model,
    feature_names=X.columns.to_list(),
    class_names=pd.Categorical(y).unique(),
    rounded=True,
    filled=True,
    fontsize=17,
)
plt.title('Decision Tree Classifier for Stellar Classification Dataset', size=30)
plt.savefig('plots/best_tree_structure.png', bbox_inches='tight', dpi=500)
plt.close()


<img src="plots/best_tree_structure.png" width="1650" title="Tree Structure">


### Análise das regras de decisão

Os principais atributos analisados nas regras geradas pela árvore acima são `redshift`, `g` e `z`. Observando a árvore, vemos que o nó raiz se divide em dois ramos, avaliando `redshift <= -0.789` e `redshift <= 0.581`.

Para o primeiro caso, a maioria das instâncias são classificadas como `STAR`, havendo uma tendência para esta classificação com `redshift <= -0.795` e `redshift <= -0.789`. Algumas exceções são observadas, com instâncias sendo classificadas como `GALAXY` de acordo com o valor de `z`.

No segundo caso, observamos predominância de classificação em `GALAXY` para `z <= 0.03`. O atributo `g` parece exercer influência na classificação em `QSO`, que ocorre para `g <= 0.053` e `g <= 0.029`.

Os nós com entropia mais baixa indicam maior pureza na classificação, ou seja, a maioria das instâncias nesses nós pertencem a uma única classe. Isso sugere uma boa separação entre as classes com base nas regras avaliadas.

Como o atributo de `redshift` é o primeiro a ser avaliado, ele exerce grande influência na decisão de separação das classes. O *redshift* (desvio para o vermelho) é uma medida-chave na astronomia para diferenciar objetos celestes. O atributo `z`, relacionado ao filtro infravermelho utilizado, está sendo utilizado para distinguir as estrelas em alguns casos onde o *redshift* é negativo, enquanto o atributo `g`, relacionado ao filtro verde, parece estar sendo utilizado para distinguir galáxias de quasars para maiores valores do *redshift*.

### Testando outros algoritmos
Vamos agora aplicar outros algoritmos ao nosso problema de classificação, a saber: 
- **Support Vector Machine (SVM)**: O SVM é um algoritmo de classificação que tenta encontrar um hiperplano que melhor separa as classes de dados. No nosso caso, vamos utilizar um kernel **RBF** transforma os dados de forma não linear para um espaço de maior dimensão, tornando-os mais facilmente separáveis. Os parâmetros que vamos usar são os seguintes:
    - `C`: responsável pela regularização. Um C maior permite mais erros de classificação, o que generaliza melhor o modelo e diminui a possibilidade de *overfitting*. Um C menor penaliza mais os erros de classificação, criando uma separação de classes mais precisa, porém pode levar ao *overfitting*.
    -  `gamma`: define como cada ponto de amostra influencia o limite de decisão, afetando sua distância de influência. Um gamma baixo faz com que pontos distantes do limite de decisão tenham uma influência maior, levando a um modelo mais suave. Um gamma alto aumenta a influência dos pontos próximos ao limite de decisão, o que pode resultar em *overfitting*.
    

- **Algoritmo Ingênuo de Bayes**: Baseado no teorema de Bayes, esse algoritmo assume que os atributos são independentes entre si. Como nossos dados são contínuos, vamos utilizar a versão **gaussiana**, assumindo que os valores seguem uma distribuição normal.

- **K-Nearest Neighbors (KNN)**: Um algoritmo de classificação que atribui a classe de um dado com base nas classes dos k vizinhos mais próximos no espaço das características. Ele é baseado inteiramente nos dados de treinamento. Recebe como parâmetro:
    - `n_neighbors`:

- **Perceptron Multi Camadas (MLP)**: É uma rede neural feedforward com pelo menos uma camada oculta. Cada camada é composta por neurônios com funções de ativação, e o MLP pode aprender funções complexas e não lineares, sendo treinado via retropropagação. Vamos usar os seguintes parâmetros:
    - `solver`: método usado para otimizar os pesos durante o treinamento. 
    - `alpha`: parâmetro de regularização L2 que ajuda a evitar overfitting, penalizando coeficientes grandes nos pesos do modelo.
    - `hidden_layer_sizes`: define o número e o tamanho das camadas ocultas na MLP. Por exemplo, (100, 50) representa duas camadas ocultas, a primeira com 100 neurônios e a segunda com 50.
    - `random_state`: define a semente do gerador de números aleatórios, garantindo que a inicialização dos pesos e o particionamento de dados sejam reprodutíveis.
    - `validation_fraction`: porcentagem dos dados de treinamento reservada para validação durante o treinamento.
    - `max_iter`: número máximo de iterações para o algoritmo de otimização

Vamos começar definindo uma função para retornar o resultado dos treinamentos num formato de tabela. A árvore que treinamos acima também será treinada novamente abaixo:

In [35]:
def models_report(
    model_list: list,
    df_split: tuple,
):
    """Pipeline to evaluate multiple models.

    Args:
        model_list (list): List of dictionaries containing the model and its
            name. 
            Example: [{'model_name': 'Decision Tree',
                      'estimator': DecisionTreeClassifier()}]
        df_split (tuple): Tuple containing the train and test dataframes.
            Example: (X_train, X_test, y_train, y_test)

    Returns:
        Pandas.DataFrame: Dataframe containing the metrics of each model.
    """
    X_train, X_test, y_train, y_test = df_split
    results_list = []
    for mdl in model_list:
        model = mdl.get('estimator')
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        model_name = mdl.get('model_name')
        results_dict = get_metrics(model_name, y_test, y_pred)
        results_list.append(results_dict)
    results_df = pd.DataFrame(
        results_list,
        columns=['model_name', 'accuracy', 'precision',
                'recall', 'f1_score'],
    )
    return results_df

Agora, vamos montar uma lista de dicionários com os modelos que vamos utilizar e passá-lo como argumento para a função `models_report`:

In [36]:
model_list = [
    {
        "model_name": "SVM",
        "estimator": SVC(kernel='rbf', C=1, gamma=0.1, random_state=42),
    },
    {
        "model_name": "Bayes ingênuo",
        "estimator": GaussianNB(),
    },
    {
        "model_name": "KNN",
        "estimator": KNeighborsClassifier(n_neighbors=3),
    },
    {
        "model_name": "MLP",
        "estimator": MLPClassifier(solver='adam', alpha=1e-3, hidden_layer_sizes=(6,), random_state=42, validation_fraction=0.2, max_iter=1000),
    },
    {
        "model_name": "Árvore de decisão",
        "estimator": DecisionTreeClassifier(criterion='entropy', max_depth=4, min_samples_leaf=1, random_state=42, ccp_alpha=0.0),
    },
]


Criando o relatório e realizando o treinamento:

In [37]:
models_report(model_list, (X_train, X_test, y_train, y_test))

,model_name,accuracy,precision,recall,f1_score
0,SVM,0.95790,0.955043,0.951502,0.952341
1,Bayes ingênuo,0.74155,0.802836,0.646010,0.613055
2,KNN,0.94390,0.943510,0.932252,0.937439
3,MLP,0.96680,0.965844,0.958274,0.961611
4,Árvore de decisão,0.96535,0.968019,0.950889,0.958834


Vemos então que a árvore de decisão possui métricas muito boas. Além disso, a árvore possui a vantagem de ser bem mais facilmente interpretável. Vamos utilizar técnicas de seleção de atributos para melhorar ainda mais esse modelo.

## Seleção de Atributos
Vamos utilizar técnicas de seleção de atributos para identificar as variáveis consideradas mais relevantes para o problema de classificação escolhido. 

In [38]:
importance = tree_model.feature_importances_
feature_names = X.columns.to_list()

plt.figure(figsize=(12, 6))

sns.barplot(x=importance, y=feature_names)
plt.title('Feature Importance')
plt.savefig('plots/feature_importance.png', bbox_inches='tight', dpi=300)
plt.close()


<img src="plots/feature_importance.png" title="Feature Importance" width="1100">

Vemos então que, exatamente como analisamos através do plot da árvore de decisão, os atributos `redshift` e `g` são os mais importantes na hora da classificação. O atributo `z` aparece como um terceiro colocado, mas vamos nos concentrar nos dois primeiros e retreinar nosso modelo utilizando-os apenas. 

In [39]:
thresholds = np.sort(importance)[-3] + 0.01
sfm = SelectFromModel(tree_model, threshold=thresholds).fit(X_train, y_train)
sfm

SelectFromModel(estimator=DecisionTreeClassifier(criterion='entropy',
                                                 max_depth=4, random_state=42),
                threshold=0.016922808652881042)

In [40]:
support = sfm.get_support()

selected_features = X.columns[sfm.get_support()]
selected_features

Index(['g', 'redshift'], dtype='object')

In [41]:
selected_df = X[selected_features]
selected_df.head()

,g,redshift
0,0.054926,0.079557
1,0.072456,0.277095
2,0.067165,0.092422
3,0.102209,0.486768
4,-0.092947,-0.630263


Realizando o treinamento:

In [42]:
X_train, X_test, y_train, y_test = train_test_split(selected_df, y, test_size=0.2, random_state=42)

fs_tree_model = DecisionTreeClassifier(criterion='entropy', max_depth=4, min_samples_leaf=1, random_state=42, ccp_alpha=0.0)
fs_tree_model.fit(X_train, y_train)
y_pred = fs_tree_model.predict(X_test)

tree_model_report(y_test, y_pred, 'fs_tree_report')

Vamos plotar uma curva de aprendizado para garantir que não ocorreu overfitting:

In [43]:
train_sizes, train_scores, test_scores = learning_curve(
    fs_tree_model, X_train, y_train, cv=5, n_jobs=-1, train_sizes=np.linspace(0.1, 1.0, 10), scoring='accuracy'
)

# Média e desvio padrão para as acurácias de treino e validação
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

# Plotando a curva de aprendizado
plt.figure(figsize=(8, 6))
plt.plot(train_sizes, train_mean, label='Acurácia de Treinamento', color='blue')
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, color='blue', alpha=0.2)
plt.plot(train_sizes, test_mean, label='Acurácia de Validação', color='green')
plt.fill_between(train_sizes, test_mean - test_std, test_mean + test_std, color='green', alpha=0.2)

plt.title('Curva de Aprendizado para Decision Tree')
plt.xlabel('Tamanho do Conjunto de Treinamento')
plt.ylabel('Acurácia')
plt.legend(loc='best')
plt.grid(True)
plt.savefig('plots/learning_curve.png', bbox_inches='tight', dpi=300)
plt.close()

<img src="plots/fs_tree_report.png" title="Tree with Feature Selection" width="600">

<img src="plots/learning_curve.png" title="Tree Learning Curve" width="750">

Nosso modelo permaneceu com bons resultados e sem overfitting!

## Interpretabilidade do modelo

Vamos plotar alguns gráficos **PDP (Partial Dependence Plot)**. Gráficos PDP mostram o efeito marginal de um ou dois atributos na predição realizada do modelo, enquanto os outros permancem fixos. É um método de interpretabilidade global agnóstico em relação a modelos (não depende deles). PDP é intuitivo e fácil de se entender, no entanto só funciona para no máximo 2 atributos para cada gráfico. No nosso caso, vamos plotar com 1 atributo por gráfico. 

In [44]:
pdp_galaxy = PartialDependenceDisplay.from_estimator(fs_tree_model, X_test, [0, 1], target='GALAXY', grid_resolution=20, random_state=42)
plt.savefig('plots/pdp_plot_galaxy.png', bbox_inches='tight', dpi=300)
plt.close()

pdp_qso = PartialDependenceDisplay.from_estimator(fs_tree_model, X_test, [0, 1], target='QSO', grid_resolution=20, random_state=42)
plt.savefig('plots/pdp_plot_qso.png', bbox_inches='tight', dpi=300)
plt.close()

pdp_star = PartialDependenceDisplay.from_estimator(fs_tree_model, X_test, [0, 1], target='STAR', grid_resolution=20, random_state=42)
plt.savefig('plots/pdp_plot_star.png', bbox_inches='tight', dpi=300)
plt.close()

Temos um para cada classe abaixo: **GALAXY**, **QSO** e **STAR** respectivamente.

<img src="plots/pdp_plot_galaxy.png" title="Galaxy class" width="600">

<img src="plots/pdp_plot_qso.png" title="Quasar class" width="600">

<img src="plots/pdp_plot_star.png" title="Star class" width="600">

Notamos novamente pelos gráficos a força dos atributos `g` e `redshift`. Algo que foi notado muito antes na árvore de decisão mostrando como esse modelo é fácil de ser interpretado. Galáxias possuem uma energia luminosa verde muito alta, e seu valor de redshift é também considerável sendo menor apenas do que os quasares, que são buracos negros supermassivos formados no inicio do universo, possuindo redshifts muito altos. As estrelas possuem o menor de ambos atributos.

## Conclusão

Testamos diversos modelos neste dataset. Concluímos que a árvore de decisão com os parâmetros especificados acima foi a melhor escolha, tanto pelo seu desempenho nas métricas, como pela sua interpretabilidade. Simplificamos ainda mais o modelo utilizando seleção de atributos (com feature importance) e PDP para tornar mais interpretável. 